# 案例：全美婴儿姓名

- 案例来源：[Chapter 4 Pipes and dplyr: the easy way of data manipulation | Machine learning in R](https://faculty.washington.edu/otoomet/machinelearning-R/pipes-dplyr.html#combining-dplyr-function)
- 数据来源：[hadley/babynames: An R package containing US baby names from the SSA](https://github.com/hadley/babynames)

In [1]:
import pandas as pd

In [2]:
# baby_names_raw.feather：1880-2017年完整数据，共1924665条.
# baby_names = pd.read_feather('data/baby_names_raw.feather')

# baby_names.csv：1995-2003年的子数据，用于演示.
baby_names = pd.read_csv('data/baby_names.csv')

In [3]:
baby_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257707 entries, 0 to 257706
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   year    257707 non-null  int64 
 1   sex     257707 non-null  object
 2   name    257707 non-null  object
 3   n       257707 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 7.9+ MB


In [4]:
baby_names.head()

,year,sex,name,n
0,1995,F,Jessica,27935
1,1995,F,Ashley,26602
2,1995,F,Emily,24380
3,1995,F,Samantha,21645
4,1995,F,Sarah,21376


## 任务一

在每一年中，找出**既有男孩使用，又有女孩使用**的名字中，最受欢迎的名字

---

思路：

- 根据年份和名字分组，筛选出出现了**2**次的名字，即**同时**被男孩和女孩使用的名字
- 计算这些名字在**一年中**被男孩使用的次数和女孩使用的次数**之和**
- 筛选出**每一年中**总次数排名第一的名字，即最受欢迎的名字

In [5]:
(
    baby_names.set_index(['year', 'name'])
    .loc[lambda df: df.groupby(['year', 'name']).size() == 2]
    .groupby(['year', 'name'])['n']
    .sum()
    .loc[lambda s: s.groupby('year').idxmax()]
    .reset_index()
)

,year,name,n
0,1995,Michael,41569
1,1996,Michael,38522
2,1997,Michael,37690
3,1998,Michael,36742
4,1999,Jacob,35424
5,2000,Jacob,34530
6,2001,Jacob,32579
7,2002,Jacob,30609
8,2003,Jacob,29677


## 任务二

分别对于男孩和女孩，筛选出**连续**入选前十的名字，并找出每个名字**最长**连续区间的长度（年份数）、开始年份和结束年份

---

思路：

- 按年份和性别分组，筛选出进入前十的名字
- 按性别和名字分组，判断这些进入前十的名字是否出现年份**间断**
- 如果出现年份**间断**，则根据**间断点**再次对名字进行分组，以确保每个组内的年份都是**连续**的
- 计算每个组的区间长度、开始年份、结束年份
- 对于出现了**间断**的名字，筛选出其连续区间长度**最长**的部分

In [6]:
(
    baby_names.loc[
        lambda df: df.groupby(['year', 'sex'])['n'].rank(ascending=False) <= 10
    ]
    .sort_values(['sex', 'name', 'year'])
    .assign(
        diff=lambda df: df.groupby(['sex', 'name'])['year'].diff().fillna(0),
        gap=lambda df: df['diff'] > 1,
        period=lambda df: df.groupby(['sex', 'name'])['gap'].cumsum(),
    )
    .groupby(['sex', 'name', 'period'])
    .agg(length=('year', 'size'), start=('year', 'first'), end=('year', 'last'))
    .loc[lambda df: df.groupby(['sex', 'name'])['length'].idxmax()]
    .reset_index()
    .sort_values('length', ascending=False, ignore_index=True)
)

,sex,name,period,length,start,end
0,F,Ashley,0,9,1995,2003
1,F,Emily,0,9,1995,2003
2,F,Hannah,0,9,1995,2003
3,F,Samantha,0,9,1995,2003
4,M,Joshua,0,9,1995,2003
5,M,Matthew,0,9,1995,2003
6,M,Christopher,0,9,1995,2003
7,M,Jacob,0,9,1995,2003
8,M,Michael,0,9,1995,2003
9,M,Andrew,0,8,1996,2003
